# 1. Rainfall Erosivity Factor Computation Using Kinetic Energy Equation

This code is developed to calculate rainfall erosivity factor (R factor) using kinetic energy equation based on **5-minute interval precipitation gauge data during 1994-2023** requested from Oklahoma Mesonet (chartman@mesonet.org). The data in the "RAIN" column is **cumulative rainfall amount with unit of inch**, which also includes **frozen precipitation** measured at the time of thaw. Each Mesonet observation contains a running accumulation of rainfall since either 6 PM CST or 7 PM CDT. As each new evening begins, the accumulated rainfall is reset to zero. The missing data are indicated by -99 or -999. More information can be seen https://mesonet.org/about/instruments.

The computation method 

## 1.1 Preparation

### 1.1.1 Filter out the missing data and classify rainfall data based on site ID (STID)
Since the missing data are indicated by -99 or -999, this code removes missing data by filtering out the data less than zero, maintaining the data greater and equal to zero.
This code also classify the rainfall data by site ID.

In [4]:

import pandas as pd
import glob
import math
import os
import numpy as np
import shutil
import arcpy
import zipfile

In [2]:

## Define the input directory for 5-minute rainfall data
path_5min_rainfall = os.path.join(os.getcwd(), "RAW")

# Define the output directory for saving storm data
path_each_station = os.path.join(os.getcwd(), "Each Station")

# Create the output directory if it doesn't exist
os.makedirs(path_each_station, exist_ok=True)

# Process each CSV file
for file_path in glob.glob(os.path.join(path_5min_rainfall, '*.csv')):
    # Load the dataset
    df = pd.read_csv(file_path)

    # Assuming the 'RAIN' column exists and filtering data
    df = df[df['RAIN'] >= 0]

    # Extract unique station IDs
    unique_stations = df['STID'].unique()

    # Loop through each unique station, filter the DataFrame, and save to a new CSV
    for station in unique_stations:
        # Filter the DataFrame for the current station
        df_station = df[df['STID'] == station][['STID', 'TIME', 'RAIN']]

        # Define the output file path using the station name, safely handle file names
        output_file_name = f"{station.replace('/', '_')}.csv"  # Replace '/' with '_' to avoid path issues
        output_file_path = os.path.join(path_each_station, output_file_name)

        # Save the filtered DataFrame to a CSV file
        df_station.to_csv(output_file_path, index=False)
        print(f"Saved data for station {station} to {output_file_path}")

print("All files have been processed and saved.")


Saved data for station ACME to f:\R factor Calculation\Each Station\ACME.csv
Saved data for station ADAX to f:\R factor Calculation\Each Station\ADAX.csv
Saved data for station ALTU to f:\R factor Calculation\Each Station\ALTU.csv
Saved data for station ALV2 to f:\R factor Calculation\Each Station\ALV2.csv
Saved data for station ANT2 to f:\R factor Calculation\Each Station\ANT2.csv
Saved data for station APAC to f:\R factor Calculation\Each Station\APAC.csv
Saved data for station ARD2 to f:\R factor Calculation\Each Station\ARD2.csv
Saved data for station ARNE to f:\R factor Calculation\Each Station\ARNE.csv
Saved data for station BEAV to f:\R factor Calculation\Each Station\BEAV.csv
Saved data for station BBOW to f:\R factor Calculation\Each Station\BBOW.csv
Saved data for station BUFF to f:\R factor Calculation\Each Station\BUFF.csv
Saved data for station BURB to f:\R factor Calculation\Each Station\BURB.csv


MemoryError: Unable to allocate 636. MiB for an array with shape (9, 9257763) and data type int64

### 1.1.2 Convert Time Format and Select Station 
The time format of the raw data from Oklahoma Mesonet is string format, which cannot be used to conduct calculations. 
The rainfall erosivity factor needs long-term measurement data, at least 10 years suggested by AH ARS (2013). Hence, this section:
1) uses the **to_datetime function** to covert the time format for computation (more information about **to_datetime function** can be learned from: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html);
2) filters out the stations with measurement periods of less than 10 years.



In [ ]:
# Input and output directories
path_each_station = os.path.join(os.getcwd(), "Each Station")

path_long_term = os.path.join(os.getcwd(), "Long-term Stations")
os.makedirs(path_long_term, exist_ok=True)

path_result = os.path.join(os.getcwd(), "Result")
os.makedirs(path_result, exist_ok=True)

# Initialize an empty list to store station-wise data recording periods
station_periods = []

# Process each CSV file in the input directory
for filename in os.listdir(path_each_station):
    if filename.endswith('.csv'):
        filepath = os.path.join(path_each_station, filename)

        # Read the CSV file into a Pandas DataFrame
        df = pd.read_csv(filepath)

        # Convert the value in "TIME" column from string format to datetime objects
        df["TIME"] = pd.to_datetime(df["TIME"], format="%Y-%m-%dT%H:%M", errors='coerce')

        # Calculate the measurement period for the station
        period = df["TIME"].max() - df["TIME"].min()  # Get the timedelta object
        
        # Calculate period in years
        period_years = period.days / 365.25  # Considering leap years
        
        # Append station and its period to the list
        station_periods.append({
            'STID': filename.split('.')[0], 
            'Start Time': df["TIME"].min(), 
            'End Time': df["TIME"].max(), 
            'Period': period,
            'Period Years': period_years
        })

        # If the period is less than 10 years, skip processing and move to the next station
        if period.days < 365 * 10:  # Check if the recording period is less than 10 years
            print(f"Skipping {filename} as data recording period is less than 10 years.")
            continue

        # Define the output path for long-term stations
        new_file_path = os.path.join(path_long_term, filename)

        # Save the filtered DataFrame to the new CSV file
        df.to_csv(new_file_path, index=False)
        print(f"Saved long-term data for {filename}")

# Convert the list of dictionaries to a DataFrame for recording periods
station_periods_df = pd.DataFrame(station_periods)

# Save the station-wise data recording periods DataFrame to a CSV file
station_periods_df.to_csv(os.path.join(path_result, 'station_periods.csv'), index=False)

Saved long-term data for ACME.csv
Saved long-term data for ADAX.csv
Saved long-term data for ALTU.csv
Saved long-term data for ALV2.csv
Saved long-term data for ANT2.csv
Saved long-term data for APAC.csv
Saved long-term data for ARD2.csv
Saved long-term data for ARNE.csv
Skipping BBOW.csv as data recording period is less than 10 years.
Saved long-term data for BEAV.csv
Saved long-term data for BESS.csv
Saved long-term data for BIXB.csv
Saved long-term data for BLAC.csv
Saved long-term data for BOIS.csv
Saved long-term data for BREC.csv
Saved long-term data for BRIS.csv
Saved long-term data for BUFF.csv
Saved long-term data for BURB.csv
Saved long-term data for BURN.csv
Saved long-term data for BUTL.csv
Saved long-term data for BYAR.csv
Saved long-term data for CAMA.csv
Saved long-term data for CARL.csv
Saved long-term data for CENT.csv
Saved long-term data for CHAN.csv
Saved long-term data for CHER.csv
Saved long-term data for CHEY.csv
Saved long-term data for CHIC.csv
Saved long-term 

In [2]:
import os
import pandas as pd

# Input and output directories
path_each_station = os.path.join(os.getcwd(), "Each Station")
path_26_year = os.path.join(os.getcwd(), "26-year Stations")
os.makedirs(path_26_year, exist_ok=True)

# Process each CSV file in the input directory
for filename in os.listdir(path_each_station):
    if filename.endswith('.csv'):
        filepath = os.path.join(path_each_station, filename)
        print(f"Processing file: {filename}")

        # Read the CSV file into a Pandas DataFrame
        try:
            df = pd.read_csv(filepath)
        except Exception as e:
            print(f"Error reading {filename}: {e}")
            continue

        # Check if 'TIME' column exists
        if "TIME" not in df.columns:
            print(f"'TIME' column missing in {filename}")
            continue

        # Convert the 'TIME' column to datetime
        df["TIME"] = pd.to_datetime(df["TIME"], format="%Y-%m-%dT%H:%M", errors='coerce')
        df = df.dropna(subset=["TIME"])  # Remove invalid timestamps

        # Check if the file has sufficient data for period calculation
        if df["TIME"].min() is pd.NaT or df["TIME"].max() is pd.NaT:
            print(f"Invalid or insufficient 'TIME' data in {filename}")
            continue

        # Calculate the measurement period in years
        period = df["TIME"].max() - df["TIME"].min()
        period_years = period.days / 365.25
        print(f"Measurement period for {filename}: {period_years} years")

        # Check if the period is greater than or equal to 26 years
        if period_years >= 26:
            new_file_path = os.path.join(path_26_year, filename)
            try:
                df.to_csv(new_file_path, index=False)
                print(f"Saved long-term data for {filename}")
            except Exception as e:
                print(f"Error saving {filename}: {e}")


Processing file: ACME.csv
Measurement period for ACME.csv: 29.95208761122519 years
Saved long-term data for ACME.csv
Processing file: ADAX.csv
Measurement period for ADAX.csv: 30.083504449007528 years
Saved long-term data for ADAX.csv
Processing file: ALTU.csv
Measurement period for ALTU.csv: 30.083504449007528 years
Saved long-term data for ALTU.csv
Processing file: ALV2.csv
Measurement period for ALV2.csv: 25.12251882272416 years
Processing file: ANT2.csv
Measurement period for ANT2.csv: 12.799452429842573 years
Processing file: APAC.csv
Measurement period for APAC.csv: 29.9958932238193 years
Saved long-term data for APAC.csv
Processing file: ARD2.csv
Measurement period for ARD2.csv: 19.939767282683093 years
Processing file: ARNE.csv
Measurement period for ARNE.csv: 30.083504449007528 years
Saved long-term data for ARNE.csv
Processing file: BBOW.csv
Measurement period for BBOW.csv: 8.131416837782341 years
Processing file: BEAV.csv
Measurement period for BEAV.csv: 30.083504449007528 y

In [6]:
# Import required libraries
import os
import pandas as pd

# Input and output directories
path_each_station = os.path.join(os.getcwd(), "Each Station")
path_filtered = os.path.join(os.getcwd(), "Filtered Stations")  # Output folder
os.makedirs(path_filtered, exist_ok=True)

# Define the required date range
start_date = pd.Timestamp("1995-01-01")
end_date = pd.Timestamp("2023-12-31")

# Process each CSV file in the input directory
for filename in os.listdir(path_each_station):
    if filename.endswith('.csv'):
        filepath = os.path.join(path_each_station, filename)

        # Read the CSV file into a Pandas DataFrame
        df = pd.read_csv(filepath)

        # Convert the value in "TIME" column from string format to datetime objects
        df["TIME"] = pd.to_datetime(df["TIME"], format="%Y-%m-%dT%H:%M", errors='coerce')

        # Filter out rows with invalid or missing dates
        df = df.dropna(subset=["TIME"])

        # Filter the data to include only rows within the specified date range
        df_filtered = df[(df["TIME"] >= start_date) & (df["TIME"] <= end_date)]

        # Check if the filtered data covers the required date range
        if not df_filtered.empty and df_filtered["TIME"].min() <= start_date and df_filtered["TIME"].max() >= end_date:
            new_file_path = os.path.join(path_filtered, filename)
            df_filtered.to_csv(new_file_path, index=False)
            print(f"Saved file with required date range: {filename}")


Saved file with required date range: ACME.csv
Saved file with required date range: ADAX.csv
Saved file with required date range: ALTU.csv
Saved file with required date range: APAC.csv
Saved file with required date range: ARNE.csv
Saved file with required date range: BEAV.csv
Saved file with required date range: BESS.csv
Saved file with required date range: BIXB.csv
Saved file with required date range: BLAC.csv
Saved file with required date range: BOIS.csv
Saved file with required date range: BREC.csv
Saved file with required date range: BRIS.csv
Saved file with required date range: BUFF.csv
Saved file with required date range: BURB.csv
Saved file with required date range: BURN.csv
Saved file with required date range: BUTL.csv
Saved file with required date range: BYAR.csv
Saved file with required date range: CAMA.csv
Saved file with required date range: CENT.csv
Saved file with required date range: CHAN.csv
Saved file with required date range: CHER.csv
Saved file with required date rang

### 1.1.3 Obtain 5-minute interval rainfall depth
Mesonet sites measured the accumulated rainfall at every 5 minutes. The column of "RAIN" in raw data requested from Mesonet is accumulated rainfall depth with unit inch.
This section is to calculate rainfall depth for each 5-minute interval and save the results with unit millimeter.

In [2]:
# Define input and output directories
path_long_term = os.path.join(os.getcwd(), "Filtered Stations")
path_Storms_Identified_By_Increas = os.path.join(os.getcwd(), "Storms_Identified_By_Increas_29")

# Create the output directory if it doesn't exist
os.makedirs(path_Storms_Identified_By_Increas, exist_ok=True)

# Search for CSV files in the input directory
pattern = os.path.join(path_long_term, '**/*.csv')
csv_files = glob.glob(pattern, recursive=True)

# Process each CSV file
for file_path in csv_files:
    df = pd.read_csv(file_path)

    # Initialize a storm marker column
    df['Storm_Marker'] = False

    # Mark rows where RAIN increases from the previous value
    for i in range(1, len(df)):
        if df.loc[i, 'RAIN'] > df.loc[i - 1, 'RAIN']:
            df.loc[i, 'Storm_Marker'] = True
            df.loc[i - 1, 'Storm_Marker'] = True  # Mark the starting point

    # Filter the DataFrame to keep only rows marked as part of a storm
    storm_df = df[df['Storm_Marker']].copy()  # Work on a copy to avoid warnings

    # Check if there are any storms identified
    if not storm_df.empty:
        # Define the new file path in the output directory
        relative_path = os.path.relpath(file_path, path_long_term)
        new_file_path = os.path.join(path_Storms_Identified_By_Increas, relative_path)

        # Create subdirectories in the output directory if they don't exist
        os.makedirs(os.path.dirname(new_file_path), exist_ok=True)

        # Save the storm data to a new CSV file
        storm_df.drop('Storm_Marker', axis=1, inplace=True)  # Remove the marker column
        storm_df.to_csv(new_file_path, index=False)

        print(f"Storm data saved for {os.path.basename(file_path)} to {new_file_path}")
    else:
        print(f"No storm identified in {os.path.basename(file_path)}.")

Storm data saved for ACME.csv to c:\Users\mengting chen\OneDrive - Oklahoma A and M System\Mengtings Research Folder\Erosion and Vulnerability Assessment\R_estimation_gauge\R_Computation\Storms_Identified_By_Increas_26\ACME.csv
Storm data saved for ADAX.csv to c:\Users\mengting chen\OneDrive - Oklahoma A and M System\Mengtings Research Folder\Erosion and Vulnerability Assessment\R_estimation_gauge\R_Computation\Storms_Identified_By_Increas_26\ADAX.csv
Storm data saved for ALTU.csv to c:\Users\mengting chen\OneDrive - Oklahoma A and M System\Mengtings Research Folder\Erosion and Vulnerability Assessment\R_estimation_gauge\R_Computation\Storms_Identified_By_Increas_26\ALTU.csv
Storm data saved for APAC.csv to c:\Users\mengting chen\OneDrive - Oklahoma A and M System\Mengtings Research Folder\Erosion and Vulnerability Assessment\R_estimation_gauge\R_Computation\Storms_Identified_By_Increas_26\APAC.csv
Storm data saved for ARNE.csv to c:\Users\mengting chen\OneDrive - Oklahoma A and M Syst

### 1.1.4 Storm Separation
This code consider that a 5-minute interval with a rainfall depth of zero that no rainfall event occurred. This code checks each row of the DataFrame to see if there is '5-min Rainfall (mm)' > 0. Encounting 5-min Rainfall is not > 0 (no rainfall events), the current storm is determined and it increments the storm_counter, repeating the process for the next storm.


In [5]:
# Define input 
path_Storms_Identified_By_Increas = os.path.join(os.getcwd(), "Storms_Identified_By_Increas_29")
# Find all CSV files in the input directory
pattern = os.path.join(path_Storms_Identified_By_Increas, '*.csv')
csv_files = glob.glob(pattern)

# Define output
path_single_storm = os.path.join(os.getcwd(), "Single_Storm_29")
# Create the output directory if it doesn't exist
os.makedirs(path_single_storm, exist_ok=True)

def process_and_save_rainfall_events(df, path_single_storm, station_id):
    """
    Identify storm events starting from 0, followed by any values that do not decrease.
    A storm ends just before the next value that is smaller or 0.
    Each storm is saved in a separate file, starting from a 0 RAIN value and ending before a decrease or the next 0.
    """
    event_number = 0
    storm_start_index = None
    in_storm = False

    for i in range(len(df) - 1):
        current_rain = df.loc[i, 'RAIN']
        next_rain = df.loc[i + 1, 'RAIN']

        # Start a new storm
        if current_rain == 0 and next_rain > 0 and not in_storm:
            storm_start_index = i
            in_storm = True
        # End the current storm
        elif in_storm and (next_rain < current_rain or next_rain == 0):
            event_number += 1
            event_df = df.loc[storm_start_index:i]
            event_filename = f"{station_id}_event_{event_number}.csv"
            save_path = os.path.join(path_single_storm, f"{station_id}", event_filename)
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            event_df.to_csv(save_path, index=False)
            in_storm = False
            storm_start_index = i + 1 if next_rain == 0 else None

    # Handle the case where the dataset ends while still in a storm
    if in_storm:
        event_number += 1
        event_df = df.loc[storm_start_index:len(df) - 1]
        event_filename = f"{station_id}_event_{event_number}.csv"
        save_path = os.path.join(path_single_storm, f"{station_id}", event_filename)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        event_df.to_csv(save_path, index=False)

    return {"station_id": station_id, "events_saved": event_number}

# Process each file
for file_path in csv_files:
    df = pd.read_csv(file_path)

    # Extract the station ID assuming 'STID' column exists
    if 'STID' in df.columns:
        station_id = df.iloc[0]['STID']  
    else:
        raise KeyError(f"The file {file_path} does not contain 'STID' column.")

    # Process and save storm events
    process_and_save_rainfall_events(df, path_single_storm, station_id)

## 1.2 Select Effective Storms
Before identify the effective storms, this code calculates the accumulated rainfall (head named as 'Cumulative RAIN (mm)') for each storm by alculating the cumulative sum of the 5-minute interval rainfall depth ('5-min Rainfall (mm)').
An effective storm is defined as a rainfall event that has little effect on water erosion since it has a low amount and intensity. In this section, the storm events with the Cumulative RAIN < 0.5 inches (12.5 mm) are deleted (AH ARS, 2013).

In [6]:
# Define input 
path_single_storm = os.path.join(os.getcwd(), "Single_Storm_29")
pattern = os.path.join(path_single_storm, '**/*.csv')
csv_files = glob.glob(pattern, recursive=True)

# Define output
path_effective_storm = os.path.join(os.getcwd(), "Effective_Storms_29")
os.makedirs(path_effective_storm, exist_ok=True)

# Dictionary to count storm events per station
storm_event_counters = {}

# Process each CSV file
for file_path in csv_files:
    df = pd.read_csv(file_path)

    # Check if cumulative rainfall exceeds 12.5 mm (0.5 inch) at any point
    if df['RAIN'].max() < 0.5:
        continue  # Skip if cumulative rainfall never exceeds 12.5 mm

    # Determine station ID (use 'STID' if available, else use directory name)
    station_id = df['STID'].iloc[0] if 'STID' in df.columns else os.path.basename(os.path.dirname(file_path))

    # Create output directory for the station if it doesn't exist
    output_dir_station = os.path.join(path_effective_storm, station_id)
    if not os.path.exists(output_dir_station):
        os.makedirs(output_dir_station)

    # Increment the storm event counter for the current station
    storm_event_counters[station_id] = storm_event_counters.get(station_id, 0) + 1

    # Construct the new file name with the updated index
    new_file_name = f"{station_id}_event_{storm_event_counters[station_id]}.csv"
    new_file_path = os.path.join(output_dir_station, new_file_name)

    # Save the filtered DataFrame to a new CSV file
    df.to_csv(new_file_path, index=False)


KeyboardInterrupt: 

## 1.3 Calculation 

### 1.3.1 Calculate Intensity and Kinetic Energy 
This section is to
1) calculate rainfall intensity for each 5-minute interval, which is an input in kinetic energy equation.
2) find the maximum 30-minute intensity, also known as the maximum precipitation amount within 30 minutes.

In [12]:
# input
path_effective_storm = os.path.join(os.getcwd(), "Effective_Storms_29")
# Pattern to match all CSV files within the input directory
pattern = os.path.join(path_effective_storm, '**/*.csv')
# Find all files matching the pattern, including in subdirectories
csv_files = glob.glob(pattern, recursive=True)

# output
path_rainfall_intensity = os.path.join(os.getcwd(), "Rainfall_intensity_29")
os.makedirs(path_rainfall_intensity, exist_ok=True)

# Function to calculate the rainfall intensity and maximum intensity for an individual storm.
def calculate_rainfall_intensity(df):
    df['RAIN (mm)'] = df['RAIN'] * 25.4
    df['Rainfall Interval (mm)'] = df['RAIN (mm)'].diff().fillna(df['RAIN (mm)'])
    df['Rainfall Intensity (mm/h)'] = df['Rainfall Interval (mm)'] / df['Interval_minutes'] * 60
    df['Unit energy (MJ/ha*mm)'] = 0.29 * (1 - 0.72 * np.exp(-0.082 * df['Rainfall Intensity (mm/h)']))
    df['Energy in Interval (MJ/ha)'] = df['Unit energy (MJ/ha*mm)'] * df['Rainfall Interval (mm)'].astype(float)
    
    # Determine the amount of rainfall in 5-Min 
    df['5-Min Rainfall (mm)'] = df['Rainfall Interval (mm)'].fillna(0)
    # Calculate the rainfall amount for each 10, 15, 20, 25, 30 min based on the amount of rainfall in 5-minute intervals:
    # 10-Min = 2 * 5 min interval rainfall
    # 15-Min = 3 * 5 min interval rainfall
    # 20-Min = 4 * 5 min interval rainfall
    # 25-Min = 5 * 5 min interval rainfall
    # 30-Min = 6 * 5 min interval rainfall
    for interval in [10, 15, 20, 25, 30]:
        # Define interval as list, and execute the code for each item
        column_name = f'{interval}-Min Rainfall (mm)'

        # DataFrame.rolling(window, min_periods=None, center=False, win_type=None, on=None, axis=_NoDefault.no_default, closed=None, step=None, method='single')
        # Provide rolling window calculations (more details in https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html).
        # min_periods=1 is a parameter in .rolling () method in pandas.
        # ensures that the rolling sum calculation returns a value even if the window is not fully populated,
        # as long as there is at least one valid data point in the window.
        df[column_name] = df['5-Min Rainfall (mm)'].rolling(window=interval//5, min_periods=1).sum()

        # Fill the first rows corresponding to the interval with 0
        # Save the calculated result in the row at {interval}-Min in f'{interval}-Min Rainfall (mm)' column
        # Assign the previous row in f'{interval}-Min Rainfall (mm)' = 0.
        # The value of this row is not produced from calculation, since the duration is less than 10-Min.
        df.loc[0:interval//5 - 1, column_name] = 0
    return df

# Process each CSV file
for file_path in csv_files:
    df = pd.read_csv(file_path)

    # # Filter rows where RAIN is less than 0.5
    # df = df[df['RAIN'] >= 0.5]
    # if df.empty:
    #     # print(f"File {os.path.basename(file_path)} ignored due to RAIN < 0.5.")
    #     continue  # Skip the rest of the loop for this file

    # # Check if the maximum RAIN in the file is less than 0.5
    # if df['RAIN'].max() < 0.5:
    #     # print(f"File {os.path.basename(file_path)} ignored due to max RAIN < 0.5.")
    #     continue  # Skip the rest of the loop for this file.
        
    # Convert the value in "TIME" column from string format to datetime objects using the pd.to_datetime() function
    df["TIME"] = pd.to_datetime(df["TIME"], format="%Y-%m-%d %H:%M:%S")
    
    # Calculate time differences and create the "duration" column
    df["Interval"] = df["TIME"].diff()

    # Convert the time interval to minutes
    df['Interval_minutes'] = df['Interval'].dt.total_seconds() / 60
    # df["Interval (hh:mm:ss)"] = df["Interval"].astype(str).str[-8:]
    
    # Calculate the rainfall intensity running the function "calculate_rainfall_intensity(df)"
    df_processed = calculate_rainfall_intensity(df)

    # Extract station name for subfolder creation
    station_id = df_processed['STID'].iloc[0]
    subfolder_path = os.path.join(path_rainfall_intensity, station_id)

    # Create subfolder if it doesn't exist
    if not os.path.exists(subfolder_path):
        os.makedirs(subfolder_path)

    # Define the new file path in the subfolder
    new_file_path = os.path.join(subfolder_path, os.path.basename(file_path))

    # Save the processed DataFrame to a new CSV file
    df_processed.to_csv(new_file_path, index=False)
    # print(f"station {station_id} File saved: {new_file_path}")

print('All done!')    

All done!


### 1.3.2 Calculate rainfall erosivity 

In [13]:
# Input and output directories
path_rainfall_intensity = os.path.join(os.getcwd(), "Rainfall_intensity_29")  # input 
path_rainfall_erosivity = os.path.join(os.getcwd(), "Rainfall_erosivity_29")  # output
os.makedirs(path_rainfall_erosivity, exist_ok=True)

# Find all the CSV files recursively in the input directory
csv_files = glob.glob(os.path.join(path_rainfall_intensity, '**/*.csv'), recursive=True)

# Function to extract station ID from the first 4 characters of the file name
def extract_station_id(file_name):
    return os.path.basename(file_name)[:4]

# List to collect summary data for all storms across all stations
summary_data = []

# Process each CSV file
for file_path in csv_files:
    df = pd.read_csv(file_path)

    # Extract components and create new columns
    df['TIME'] = pd.to_datetime(df['TIME'], format="%Y-%m-%d %H:%M:%S", errors='coerce')
    df['Month'] = df['TIME'].dt.month
    df['Day'] = df['TIME'].dt.day
    df['Year'] = df['TIME'].dt.year
    df['Hour'] = df['TIME'].dt.hour
    df['Minute'] = df['TIME'].dt.minute

    Total_rainfall_inch = df['RAIN'].max()
    Total_rainfall_mm = df['RAIN (mm)'].max()
    Time = df['TIME']

    # If the duration of a storm is less than 30 min, the I_30 is twice the amount of rain (AH 703).
    rainfall_to_max_30_SI = df['RAIN (mm)'].max() * 2

    # if the maximum 30 min rainfall is not equal to 0:
    if df['30-Min Rainfall (mm)'].max() != 0:

        # finds the index of the row with the maximum value in the "30-Min Rainfall (mm)"
        max_row_SI = df['30-Min Rainfall (mm)'].idxmax()

        # then retrieves the value from the 'RAIN (mm)' column at that index
        rainfall_to_max_30_SI = df.at[max_row_SI, 'RAIN (mm)']

        # calculate the maximum rainfall intensity with in 30 minutes
        max_30_min_rainfall_intensity_SI = rainfall_to_max_30_SI * 60 / 30

    # if the maximum 30-Min rainfall is = 0, return the default: the I_30 is twice the amount of rain
    else:
        max_30_min_rainfall_intensity_SI = rainfall_to_max_30_SI

    # calculate the total energy of a storm
    total_energy_SI = df['Energy in Interval (MJ/ha)'].sum()
    
    # calculate rainfall erosivity of a storm
    rainfall_erosivity_SI = total_energy_SI * max_30_min_rainfall_intensity_SI

    # Collect summary for each storm
    for station_id in df['STID'].unique():
        summary_data.append({
            "Storm File": os.path.basename(file_path),
            "Station ID": station_id,
            "TIME":Time,
            "Total Rainfall (inch)":Total_rainfall_inch,
            "Total Rainfall (mm)": Total_rainfall_mm,
            "Rainfall @ Max 30-Min (mm)": rainfall_to_max_30_SI,
            "Max 30-Min Intensity (mm/h)": max_30_min_rainfall_intensity_SI,
            "Total Energy (MJ/ha)": total_energy_SI,
            "Rainfall Erosivity ((MJ-mm)/(ha-hr))": rainfall_erosivity_SI,
            "Day": df['Day'].iloc[0],
            "Month": df['Month'].iloc[0],
            "Year": df['Year'].iloc[0],
        })

# Convert summary data to DataFrame and save for each station
summary_df = pd.DataFrame(summary_data)
unique_stations = summary_df['Station ID'].unique()

for station_id in unique_stations:
    station_summary = summary_df[summary_df['Station ID'] == station_id]
    output_file_path = os.path.join(path_rainfall_erosivity, f"{station_id.replace('/', '_')}.csv")
    station_summary.to_csv(output_file_path, index=False)
    # print(f"Saved erosivity data for station {station_id} to {output_file_path}")

### 1.3.3 Calculate Average Annual/Monthly R factor

In [8]:
import pandas as pd
import glob
import os

# Input directory
path_rainfall_intensity = os.path.join(os.getcwd(), "Daily_rainfall_gauge_29")  # Input folder containing CSV files

# Output directory
path_result = os.path.join(os.getcwd(), "Result_29")  # Output folder for results
os.makedirs(path_result, exist_ok=True)

# Initialize an empty DataFrame to hold combined data
combined_data = pd.DataFrame()

# Load and combine all files
csv_files = glob.glob(os.path.join(path_rainfall_intensity, '*.csv'), recursive=True)
for file in csv_files:
    df = pd.read_csv(file)
    df['TIME'] = pd.to_datetime(df['TIME'], format='%Y-%m-%d')  # Parse dates
    df['Year'] = df['TIME'].dt.year
    df['Month'] = df['TIME'].dt.month
    combined_data = pd.concat([combined_data, df], ignore_index=True)

# Group by Month and Year to calculate monthly totals
monthly_totals = combined_data.groupby(['STID', 'Year', 'Month'])['24hr RAIN (mm)'].sum().reset_index()

# Calculate average monthly rainfall (over 29 years)
average_monthly = monthly_totals.groupby(['STID', 'Month'])['24hr RAIN (mm)'].mean().reset_index()
average_monthly.rename(columns={'24hr RAIN (mm)': 'Average Monthly Rainfall (mm)'}, inplace=True)

# Pivot table to have months as columns
average_monthly_pivot = average_monthly.pivot(index='STID', columns='Month', values='Average Monthly Rainfall (mm)')
average_monthly_pivot.columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Reset index for a clean output
average_monthly_pivot.reset_index(inplace=True)

# Calculate annual totals
annual_totals = monthly_totals.groupby(['STID', 'Year'])['24hr RAIN (mm)'].sum().reset_index()

# Calculate average annual rainfall (over 29 years)
average_annual = annual_totals.groupby('STID')['24hr RAIN (mm)'].mean().reset_index()
average_annual.rename(columns={'24hr RAIN (mm)': 'Average Annual Rainfall (mm)'}, inplace=True)

# Save results to the output folder
average_monthly_pivot.to_csv(os.path.join(path_result, "average_monthly_rainfall.csv"), index=False)
average_annual.to_csv(os.path.join(path_result, "average_annual_rainfall.csv"), index=False)

print(f"Processing complete. Results saved in: {path_result}")


Processing complete. Results saved in: c:\Users\mengting chen\OneDrive - Oklahoma A and M System\Mengtings Research Folder\Erosion and Vulnerability Assessment\R_estimation_gauge\R_Computation\Result_29


In [ ]:
import os
import glob
import pandas as pd
import calendar

# Output directory
path_result = os.path.join(os.getcwd(), "Result_29")
os.makedirs(path_result, exist_ok=True)

# Input directory
path_rainfall_erosivity = os.path.join(os.getcwd(), "Rainfall_erosivity_29")

# Find all CSV files in the Rainfall Erosivity folder
csv_files = glob.glob(os.path.join(path_rainfall_erosivity, '*.csv'), recursive=True)

# Generate a list of month abbreviations (Jan, Feb, ..., Dec)
month_names = [calendar.month_abbr[i] for i in range(1, 13)]

# Initialize a list to hold data for each station
station_results = []

for file in csv_files:
    # Extract station name from the filename
    station_name = os.path.basename(file).replace('.csv', '')
    
    # Read the current file
    data = pd.read_csv(file)

    # Define the long-term period
    Period = 29  

    # Calculate annual R factor and precipitation 
    annual_r_factor = data['Rainfall Erosivity ((MJ-mm)/(ha-hr))'].sum() / Period
    annual_P_inch = data['Total Rainfall (inch)'].sum() / Period
    annual_P_mm = data['Total Rainfall (mm)'].sum() / Period

    # Calculate annual rainfall erosivity density
    annual_density = (
        annual_r_factor / annual_P_mm if annual_P_mm > 0 else 0
    )
    
    # Initialize dictionaries for monthly calculations
    monthly_r_factors = {}
    monthly_P_inch = {}
    monthly_P_mm = {}
    monthly_contributions = {}
    monthly_erosivity_density = {}
    
    for month_idx, month_name in enumerate(month_names):
        # Filter data for the current month
        monthly_data = data[data['Month'] == month_idx+1]
        
        # Monthly sums
        monthly_r_sum = monthly_data['Rainfall Erosivity ((MJ-mm)/(ha-hr))'].sum()
        monthly_P_inch_sum = monthly_data['Total Rainfall (inch)'].sum()
        monthly_P_mm_sum = monthly_data['Total Rainfall (mm)'].sum()
        
        # Calculate monthly R factor, precipitation, and contributions
        monthly_r_factors[f'{month_name} R'] = monthly_r_sum / Period
        monthly_P_inch[f'{month_name} P(in)'] = monthly_P_inch_sum / Period
        monthly_P_mm[f'{month_name} P(mm)'] = monthly_P_mm_sum / Period
        monthly_contributions[f'{month_name} R contribution (%)'] = (
            (monthly_r_sum / (Period * annual_r_factor) * 100) if annual_r_factor > 0 else 0
        )
        
        # Calculate monthly rainfall erosivity density
        monthly_erosivity_density[f'{month_name} R Density ((MJ-mm)/(ha-hr-mm))'] = (
            monthly_r_sum / monthly_P_mm_sum if monthly_P_mm_sum > 0 else 0
        )
    
    # Create a dictionary for the station with all calculated values
    station_results.append({
        'stid': station_name, 
        'Period (Years)': Period,
        'Annual R Factor': annual_r_factor, 
        'Annual P (inch)': annual_P_inch,
        'Annual P (mm)': annual_P_mm,
        'Annual R Density ((MJ-mm)/(ha-hr-mm))': annual_density,
        **monthly_r_factors,
        **monthly_P_inch,
        **monthly_P_mm,
        **monthly_contributions,
        **monthly_erosivity_density
    })

# Convert the list of dictionaries to a DataFrame
station_results_df = pd.DataFrame(station_results)

# Save the station-level results to a CSV file
station_results_df.to_csv(os.path.join(path_result, 'R_factor_results.csv'), index=False)

# Initialize a list for state results and calculate the average monthly R factor, contributions, and densities
state_results = []
state_density_results = []

for month_name in month_names:
    # State-level averages
    avg_r = station_results_df[f'{month_name} R'].mean()
    avg_contribution = station_results_df[f'{month_name} R contribution (%)'].mean()
    avg_density = station_results_df[f'{month_name} R Density ((MJ-mm)/(ha-hr-mm))'].mean()
    
    state_results.append({
        'Month': month_name, 
        'Monthly R Average': avg_r,
        'Mean Contribution (%)': avg_contribution
    })
    
    state_density_results.append({
        'Month': month_name,
        'Mean R Density ((MJ-mm)/(ha-hr-mm))': avg_density
    })

# Calculate average annual rainfall erosivity density across stations
avg_annual_density = station_results_df['Annual R Density ((MJ-mm)/(ha-hr-mm))'].mean()

# Save the average annual density to a CSV file
state_density_results.append({
    'Month': 'Annual',
    'Mean R Density ((MJ-mm)/(ha-hr-mm))': avg_annual_density
})

# Convert the state results lists to DataFrames
state_results_df = pd.DataFrame(state_results)
state_density_df = pd.DataFrame(state_density_results)

# Save the state results to CSV files
state_results_df.to_csv(os.path.join(path_result, 'state_monthly_R_averages.csv'), index=False)
state_density_df.to_csv(os.path.join(path_result, 'state_monthly_R_density_averages.csv'), index=False)

print("Processing complete. Results saved to:", path_result)


# 2 Save the Calculated R to Mesonet Station file

Operation this code require to: 
Installing arcpy package to work with jupyter notebook (The installation instraction step by step can be learnt from https://www.youtube.com/watch?v=DmqtYJ-liKU).

In Part II have obtained results including long term average annual R factor, monthly R factor, and monthly contribution to annaul R factor for each site where recording precipiation over 10 years. However, those data are saved in "R_factor_results.**csv**" file. For geo-processing, this section save those data in **existed shapefile (named as "ok_mesonet_sites_all_20210715.shp")** recorded general information like station ID, station name, station coordinates, and elevation, etc., produced by Mesonet (https://mesonet.org/index.php/site/sites/mesonet_sites). 

This cell is using **AddJoin Tool** (https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/add-join.htm) to save the calculated results in .csv into shapefile based on a common field, "stid", in this case. JoinField Tool also can join a layer to another layer or table based on a common field. However, the JoinField Tool will join a layer permanently, which can change the original shapefile. The original shapefile is not expected to be change, so that AddJoin Tool is applied in this case since the this tool joins a layer temporarily. According to copy the temporary layer, the joined layer can be saved as a permanet layer without changing the original layer.  

In [12]:


# Path to your ZIP file
zip_file_path = os.path.join(os.getcwd(), "mesonet_sites_shape.zip")
# zip_file_path = os.path.join(base_dir,"Geoprocess_inputs" ,"mesonet_sites_shape.zip")

# Destination directory where the contents of the zip will be extracted
extract_to_directory =os.path.join(os.getcwd(),"Geoprocess_inputs")

# Ensure the destination directory exists
os.makedirs(extract_to_directory, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)

# Paths to the shapefile and the join table CSV
shapefile_path = os.path.join(os.getcwd(),"Geoprocess_inputs", "mesonet_sites_shape","ok_mesonet_sites_all_20210715.shp")
join_table = os.path.join(os.getcwd(), "Result_29", "R_factor_results.csv")
output_dir = os.path.join(os.getcwd(), "shapefile_outputs")

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

arcpy.env.workspace = output_dir
arcpy.env.overwriteOutput = True

# The qualifiedFieldNames environment is used by Copy Features when persisting the join field names.
arcpy.env.qualifiedFieldNames = False


# Process: Create a Feature Layer from the shapefile
temp_layer_sites = "mesonet_sites_layer"
arcpy.MakeFeatureLayer_management(shapefile_path, temp_layer_sites)
print("Temporary layer created.")

# Process: Add Join
joined_table_temp_layer = arcpy.management.AddJoin(temp_layer_sites, "stid", join_table, "stid")
print("JoinField operation completed.")

# Process: Copy Features to new shapefile
joined_shapefile_path = os.path.join(output_dir, "MesoSite_R.shp")
arcpy.CopyFeatures_management(joined_table_temp_layer, joined_shapefile_path)
print("New shapefile with joined data created.")

# Process: List and print field names in the new shapefile
fields = arcpy.ListFields(joined_shapefile_path)
for field in fields:
    print(field.name)

# Process: Remove records where Annual R Factor = 0
with arcpy.da.UpdateCursor(joined_shapefile_path, ['Annual_R_F']) as cursor:
    for row in cursor:
        if row[0] == 0:
            cursor.deleteRow()
print("Records with R factor = 0 removed.")



Temporary layer created.
JoinField operation completed.
New shapefile with joined data created.
FID
Shape
stnm
stid
name
city
rang
cdir
cnty
nlat
elon
elev
cdiv
clas
WCR05
WCS05
A05
N05
BULK5
GRAV5
SAND5
SILT5
CLAY5
TEXT5
WCR10
WCS10
A10
N10
WCR25
WCS25
A25
N25
BULK25
GRAV25
SAND25
SILT25
CLAY25
TEXT25
WCR60
WCS60
A60
N60
BULK60
GRAV60
SAND60
SILT60
CLAY60
TEXT60
WCR75
WCS75
A75
N75
BULK75
GRAV75
SAND75
SILT75
CLAY75
TEXT75
datc
datd
stid_1
Period__Ye
Annual_R_F
Annual_P__
Annual_P_1
Annual_R_D
Jan_R
Feb_R
Mar_R
Apr_R
May_R
Jun_R
Jul_R
Aug_R
Sep_R
Oct_R
Nov_R
Dec_R
Jan_P_in_
Feb_P_in_
Mar_P_in_
Apr_P_in_
May_P_in_
Jun_P_in_
Jul_P_in_
Aug_P_in_
Sep_P_in_
Oct_P_in_
Nov_P_in_
Dec_P_in_
Jan_P_mm_
Feb_P_mm_
Mar_P_mm_
Apr_P_mm_
May_P_mm_
Jun_P_mm_
Jul_P_mm_
Aug_P_mm_
Sep_P_mm_
Oct_P_mm_
Nov_P_mm_
Dec_P_mm_
Jan_R_cont
Feb_R_cont
Mar_R_cont
Apr_R_cont
May_R_cont
Jun_R_cont
Jul_R_cont
Aug_R_cont
Sep_R_cont
Oct_R_cont
Nov_R_cont
Dec_R_cont
Jan_R_Dens
Feb_R_Dens
Mar_R_Dens
Apr_R_Dens
May_R_Dens
J